In [4]:
import os
from PIL import Image, ImageFile
import tensorflow as tf
import tensorflow_io as tfio

ImageFile.LOAD_TRUNCATED_IMAGES = True



def is_valid_tif_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except Exception as e:
        print(f"Invalid .tif image: {image_path}, Error: {e}")
        return False


def clean_tif_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if not file.lower().endswith(".tif"):
                continue
            file_path = os.path.join(root, file)
            if not is_valid_tif_image(file_path):
                print(f"Removing invalid .tif image: {file_path}")
                os.remove(file_path)


def convert_tif_to_png(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(".tif"):
                file_path = os.path.join(root, file)
                output_path = file_path.rsplit(".", 1)[0] + ".png"
                try:
                    with Image.open(file_path) as img:
                        img.save(output_path, format="PNG")
                        print(f"Converted: {file_path} -> {output_path}")
                except Exception as e:
                    print(f"Failed to convert {file_path}, Error: {e}")



image_dir = "/Users/desperate/Desktop/dataset"


clean_tif_images(image_dir)
convert_tif_to_png(image_dir)

train_dataset = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels="inferred",          
    label_mode="binary",        
    image_size=(224, 224),      
    batch_size=32,              
    validation_split=0.2,       
    subset="training",          
    seed=123                    
)


validation_dataset = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels="inferred",
    label_mode="binary",
    image_size=(224, 224),
    batch_size=32,
    validation_split=0.2,       
    subset="validation",        
    seed=123
)



data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])


train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
)


base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

Converted: /Users/desperate/Desktop/dataset/real/102_6.tif -> /Users/desperate/Desktop/dataset/real/102_6.png
Converted: /Users/desperate/Desktop/dataset/real/106_2.tif -> /Users/desperate/Desktop/dataset/real/106_2.png
Converted: /Users/desperate/Desktop/dataset/real/104_1.tif -> /Users/desperate/Desktop/dataset/real/104_1.png
Converted: /Users/desperate/Desktop/dataset/real/106_3.tif -> /Users/desperate/Desktop/dataset/real/106_3.png
Converted: /Users/desperate/Desktop/dataset/real/102_7.tif -> /Users/desperate/Desktop/dataset/real/102_7.png
Converted: /Users/desperate/Desktop/dataset/real/102_5.tif -> /Users/desperate/Desktop/dataset/real/102_5.png
Converted: /Users/desperate/Desktop/dataset/real/106_1.tif -> /Users/desperate/Desktop/dataset/real/106_1.png
Converted: /Users/desperate/Desktop/dataset/real/104_3.tif -> /Users/desperate/Desktop/dataset/real/104_3.png
Converted: /Users/desperate/Desktop/dataset/real/104_2.tif -> /Users/desperate/Desktop/dataset/real/104_2.png
Converted:

In [46]:
import os
from PIL import Image, ImageFile
import tensorflow as tf
import tensorflow_io as tfio

ImageFile.LOAD_TRUNCATED_IMAGES = True

def is_valid_tif_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except Exception as e:
        print(f"Invalid .tif image: {image_path}, Error: {e}")
        return False


def clean_tif_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if not file.lower().endswith(".tif"):
                continue
            file_path = os.path.join(root, file)
            if not is_valid_tif_image(file_path):
                print(f"Removing invalid .tif image: {file_path}")
                os.remove(file_path)


def convert_tif_to_png(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(".tif"):
                file_path = os.path.join(root, file)
                output_path = file_path.rsplit(".", 1)[0] + ".png"
                try:
                    with Image.open(file_path) as img:
                        img.save(output_path, format="PNG")
                        print(f"Converted: {file_path} -> {output_path}")
                except Exception as e:
                    print(f"Failed to convert {file_path}, Error: {e}")



image_dir = "/Users/desperate/Desktop/dataset1"  
clean_tif_images(image_dir)
convert_tif_to_png(image_dir)


test_dataset = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels="inferred",          
    label_mode="binary",        
    image_size=(224, 224),      
    batch_size=32,              
    shuffle=False               
)


print("Evaluating the model on the test dataset...")
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Loss: {test_loss:.2f}")


def predict_image(model, image_path):
    try:
        img = Image.open(image_path).resize((224, 224))  
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0  
        img_array = tf.expand_dims(img_array, axis=0)  

        prediction = model.predict(img_array)[0][0]
        if prediction > 0.5:
            return f"Fake Fingerprint (Confidence: {prediction:.2f})"
        else:
            return f"Real Fingerprint (Confidence: {1 - prediction:.2f})"
    except Exception as e:
        return f"Error processing image {image_path}: {e}"


test_image_path = "/Users/desperate/Desktop/dataset1/fake/1__M_Left_index_finger_Zcut 2.BMP"  
result = predict_image(model, test_image_path)
print(result)

Converted: /Users/desperate/Desktop/dataset1/real/102_6.tif -> /Users/desperate/Desktop/dataset1/real/102_6.png
Converted: /Users/desperate/Desktop/dataset1/real/106_2.tif -> /Users/desperate/Desktop/dataset1/real/106_2.png
Converted: /Users/desperate/Desktop/dataset1/real/104_1.tif -> /Users/desperate/Desktop/dataset1/real/104_1.png
Converted: /Users/desperate/Desktop/dataset1/real/106_3.tif -> /Users/desperate/Desktop/dataset1/real/106_3.png
Converted: /Users/desperate/Desktop/dataset1/real/102_7.tif -> /Users/desperate/Desktop/dataset1/real/102_7.png
Converted: /Users/desperate/Desktop/dataset1/real/102_5.tif -> /Users/desperate/Desktop/dataset1/real/102_5.png
Converted: /Users/desperate/Desktop/dataset1/real/106_1.tif -> /Users/desperate/Desktop/dataset1/real/106_1.png
Converted: /Users/desperate/Desktop/dataset1/real/104_3.tif -> /Users/desperate/Desktop/dataset1/real/104_3.png
Converted: /Users/desperate/Desktop/dataset1/real/104_2.tif -> /Users/desperate/Desktop/dataset1/real/10